In [1]:
!apt install mongodb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
mongodb is already the newest version (1:3.6.3-0ubuntu1.4).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [2]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [3]:
!pip install pymongo


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from pymongo import MongoClient
from bs4 import BeautifulSoup as bs
import requests
from time import  sleep
import json

In [5]:
url_hh = 'https://novosibirsk.hh.ru/search/vacancy?text=python&from=suggest_post&area=4&hhtmFrom=vacancy_search_list'

headers={
    'User-Agent':	'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
}
def hh_parsing():
  vacancy_list_hh=[]
  for page in range(200):
    sleep(3) 
    url=url_hh+f'&page={page}'
    response_hh=requests.get(url, headers=headers)
    dom_hh = bs(response_hh.content, 'html.parser')
    vacancies=dom_hh.find_all('div', {'class': 'vacancy-serp-item-body__main-info'})

    vacancy_list=[]
    for vacancy in vacancies:  
      vacancy_name=vacancy.find('a', {'class': 'serp-item__title'}).text
      vacancy_link=vacancy.find('a', {'class': 'serp-item__title'})['href']
      vacancy_salary=vacancy.find('span', {'class': 'bloko-header-section-3'})
      vacancy_employee=vacancy.find('a', {'class': 'bloko-link bloko-link_kind-tertiary'}).text.replace('\xa0', '')
      
      if vacancy_salary:
        vacancy_salary=vacancy_salary.text.replace('\u202f', '').split(' ')       
        for elements in vacancy_salary:
          if len(vacancy_salary)==4:
            salary_min=vacancy_salary[0]
            salary_max=vacancy_salary[2]
            salary_currency=vacancy_salary[-1]
          elif len(vacancy_salary)==3:
            salary_min=vacancy_salary[1]
            salary_currency=vacancy_salary[-1]
      else:
        salary_min, salary_max, salary_currency=None, None, None


      vacancy_dict={
          'vacancy_name': vacancy_name,
          'vacancy_link': vacancy_link,
          'salary_min': salary_min,
          'salary_max': salary_max,
          'salary_currency': salary_currency,
          'vacancy_employee': vacancy_employee,
          'resource': 'novosibirsk.hh.ru',
    
      }
      vacancy_list.append(vacancy_dict)
    
    if len(vacancy_list)>1:
      vacancy_list_hh.append(vacancy_list)
    else:
      return vacancy_list_hh
    

just_parsed_vacancies=hh_parsing()

In [6]:
def number_of_just_parsed_vacansies(lists_of_just_parsed_vacancies):
  
  number_of_just_parsed_vacansies=0
  for lists in just_parsed_vacancies:
    for vacancies in lists:
      number_of_just_parsed_vacansies+=1
  return number_of_just_parsed_vacansies

number_of_just_parsed_vacansies(just_parsed_vacancies)
  

100

In [7]:
vacancy_list_hh='/content/vacancy_list_hh.json'

with open(vacancy_list_hh) as f:
    vacancy_hh = json.load(f)

client=MongoClient()
db=client.vacancy_hh

for listst_from_vacancy_hh in vacancy_hh:
  db.vacancy_hh.insert_many(listst_from_vacancy_hh)

downloaded_number_of_vacancies=len(list(db.vacancy_hh.find()))
downloaded_number_of_vacancies

200

In [8]:
def check_and_update_db(list_vacancies_need_to_chek_and_add_to_db):
  for list_vacancies in list_vacancies_need_to_chek_and_add_to_db:
    for vacancies in list_vacancies:
      if not db.vacancy_hh.find({'vacancy_link': vacancies['vacancy_link'], 'vacancy_employee': vacancies['vacancy_employee']}):
        db.vacancy_hh.insert_one(vacancies)
      

check_and_update_db(just_parsed_vacancies)

In [9]:
def how_much_vacancies_added_into_db():
  return len(list(db.vacancy_hh.find()))-downloaded_number_of_vacancies

In [10]:
print(f'There was added {how_much_vacancies_added_into_db()} new vacancies')

There was added 0 new vacancies
